In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew 
from scipy.special import boxcox1p
from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [2]:
train=pd.read_csv("~/Pet_Projects/House_price_predictions/house-prices-raw/train.csv")
test=pd.read_csv("~/Pet_Projects/House_price_predictions/house-prices-raw/test.csv")

In [3]:
train_x=train.drop(columns=['Id', 'SalePrice'])
train_y=train['SalePrice']
test_x=test.drop(columns=['Id'])
combine=pd.concat([train_x,test_x],ignore_index=True)
len(combine.columns)

79

In [4]:
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [5]:
a=combine.isnull().sum()/len(combine)
a=a[a>0.45]
a.sort_values(ascending=False)


PoolQC         0.996574
MiscFeature    0.964029
Alley          0.932169
Fence          0.804385
FireplaceQu    0.486468
dtype: float64

In [6]:
combine=combine.drop(columns=['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'])

In [7]:
corr=combine.corr().unstack().abs()
corr=corr[corr>0.6]
corr.sort_values(ascending=False).drop_duplicates()

YrSold        YrSold          1.000000
GarageCars    GarageArea      0.889700
GarageYrBlt   YearBuilt       0.834812
GrLivArea     TotRmsAbvGrd    0.808354
1stFlrSF      TotalBsmtSF     0.801670
BedroomAbvGr  TotRmsAbvGrd    0.669737
2ndFlrSF      GrLivArea       0.655085
YearRemodAdd  GarageYrBlt     0.652365
BsmtFinSF1    BsmtFullBath    0.638847
GrLivArea     FullBath        0.630286
YearRemodAdd  YearBuilt       0.612235
2ndFlrSF      HalfBath        0.611362
GarageCars    OverallQual     0.600744
dtype: float64

In [8]:
imp = IterativeImputer(missing_values=np.nan, sample_posterior=False, 
                                 max_iter=10, tol=0.001, 
                                 n_nearest_features=4, initial_strategy='median')
new_array=imp.fit_transform(combine.select_dtypes(include=np.number))
new_data=pd.DataFrame(new_array,columns=(combine.select_dtypes(include=np.number)).columns,index=combine.index)
new_data

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,608.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,642.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,836.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160.0,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0
2915,160.0,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,...,286.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,4.0,2006.0
2916,20.0,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,...,576.0,474.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0
2917,85.0,62.0,10441.0,5.0,5.0,1992.0,1992.0,0.0,337.0,0.0,...,0.0,80.0,32.0,0.0,0.0,0.0,0.0,700.0,7.0,2006.0


In [9]:
new_data_cat=combine.select_dtypes(exclude=np.number)
new_data_cat.fillna(value='No',inplace=True)

/home/vishesh51/env/lib/python3.8/site-packages/pandas/core/frame.py:4321: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [10]:
orde=OrdinalEncoder()
new_cat_array=orde.fit_transform(new_data_cat)
new_cat_data=pd.DataFrame(new_cat_array,columns=new_data_cat.columns,index=new_data_cat.index)
new_cat_data

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,4.0,1.0,3.0,3.0,0.0,4.0,0.0,5.0,2.0,2.0,...,5.0,2.0,7.0,1.0,2.0,5.0,5.0,2.0,9.0,4.0
1,4.0,1.0,3.0,3.0,0.0,2.0,0.0,24.0,1.0,2.0,...,5.0,4.0,7.0,1.0,2.0,5.0,5.0,2.0,9.0,4.0
2,4.0,1.0,0.0,3.0,0.0,4.0,0.0,5.0,2.0,2.0,...,5.0,2.0,7.0,1.0,2.0,5.0,5.0,2.0,9.0,4.0
3,4.0,1.0,0.0,3.0,0.0,0.0,0.0,6.0,2.0,2.0,...,5.0,2.0,7.0,5.0,3.0,5.0,5.0,2.0,9.0,0.0
4,4.0,1.0,0.0,3.0,0.0,2.0,0.0,15.0,2.0,2.0,...,5.0,2.0,7.0,1.0,2.0,5.0,5.0,2.0,9.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,5.0,1.0,3.0,3.0,0.0,4.0,0.0,10.0,2.0,2.0,...,5.0,4.0,7.0,6.0,1.0,3.0,3.0,2.0,9.0,4.0
2915,5.0,1.0,3.0,3.0,0.0,4.0,0.0,10.0,2.0,2.0,...,5.0,4.0,7.0,4.0,3.0,5.0,5.0,2.0,9.0,0.0
2916,4.0,1.0,3.0,3.0,0.0,4.0,0.0,11.0,2.0,2.0,...,5.0,4.0,7.0,5.0,3.0,5.0,5.0,2.0,9.0,0.0
2917,4.0,1.0,3.0,3.0,0.0,4.0,0.0,11.0,2.0,2.0,...,5.0,4.0,7.0,6.0,1.0,3.0,3.0,2.0,9.0,4.0


In [11]:
new_onehot_data=pd.get_dummies(combine.select_dtypes(exclude=np.number),dummy_na=True)

In [12]:
new_combine=pd.concat([new_data,new_cat_data],axis=1)

In [ ]:
from sklearn.preprocessing import FunctionTransformer
scaler = FunctionTransformer(np.log1p)
scaled_x = scaler.fit_transform(new_combine)
scaled_y= scaler.fit_transform(train_y.values.reshape(-1,1))
x_train=scaled_x[0:1460]
x_predict=scaled_x[1460:]
from sklearn.model_selection import train_test_split
x_training, x_cv, y_training, y_cv = train_test_split(x_train,scaled_y,test_size=.3, random_state=42)


In [ ]:
model3 = LinearRegression()
model3.fit(x_training, y_training)
print(model3.score(x_cv,y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('LinearRegression.csv',index=False)

In [ ]:
model4 = LinearRegression(normalize=True)
model4.fit(x_training, y_training)
model4.score(x_cv,y_cv)
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('LinearRegression2.csv',index=False)

In [13]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer()
scaled_x = scaler.fit_transform(new_combine)
scaled_y= scaler.fit_transform(train_y.values.reshape(-1,1))
x_train=scaled_x[0:1460]
x_predict=scaled_x[1460:]
from sklearn.model_selection import train_test_split
x_training, x_cv, y_training, y_cv = train_test_split(x_train,scaled_y,test_size=.3, random_state=42)

In [ ]:
len(x_train)

In [ ]:
from sklearn import svm
model3 = svm.SVR(C=1, kernel= 'linear')
model3.fit(x_training, np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('SVR.csv',index=False)

In [ ]:
from sklearn import svm
model3 = svm.NuSVR(C=1, gamma='auto', kernel= 'rbf')
model3.fit(x_training, np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('NuSVR.csv',index=False)

In [ ]:
from sklearn import svm
model3 = svm.LinearSVR(C=1)
model3.fit(x_training, np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('LinearSVR.csv',index=False)

In [ ]:
from sklearn.kernel_ridge import KernelRidge
model3 = KernelRidge(alpha=1, kernel='poly', degree= 40, coef0=100)
model3.fit(x_train, np.ravel(scaled_y))
print(model3.score(x_cv,np.ravel(y_cv)))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('KernelRidge.csv',index=False)

In [ ]:
y_predict

In [ ]:
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('KernelRidge.csv',index=False)


In [ ]:
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
model3 = SGDRegressor(loss='huber', l1_ratio= 0.1, penalty='l1')
model3.fit(x_training,np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('SGD.csv',index=False)

In [ ]:
np.asarray(np.ravel(y_training), dtype='float64').dtype

In [ ]:
from sklearn import tree
model3 = tree.DecisionTreeRegressor(max_features='auto')
model3.fit(x_training,np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('DecisiontreeRegressor.csv',index=False)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn import svm
model3 = BaggingRegressor(base_estimator=svm.NuSVR(C=1, gamma='auto', kernel= 'rbf'))
model3.fit(x_training,np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('bagging.csv',index=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model3 = RandomForestRegressor(n_estimators=100)
model3.fit(x_training,np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('RandomForestRegressor.csv',index=False)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model3 = ExtraTreesRegressor(n_estimators=100, max_depth=None, criterion='mae')
model3.fit(x_training,np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('ExtraTreesRegressor.csv',index=False)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.kernel_ridge import KernelRidge
model3 = AdaBoostRegressor(base_estimator=KernelRidge(), n_estimators=100)
model3.fit(x_training,np.ravel(y_training))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('AdaBoostRegressor.csv',index=False)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model3 = GradientBoostingRegressor(loss='lad', n_estimators=100,criterion='mae')
model3.fit(x_train,np.ravel(scaled_y))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('LinearSVR.csv',index=False)

In [ ]:
model3.score(x_cv,np.ravel(y_cv))

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import VotingRegressor
model3 = VotingRegressor([('lr', GradientBoostingRegressor(loss='lad', n_estimators=100,criterion='mae')), ('rf', KernelRidge(alpha=1, kernel='poly', degree= 4, coef0=2)),('sf',ExtraTreesRegressor(n_estimators=100, max_depth=None, criterion='mae'))])
model3.fit(x_train,np.ravel(scaled_y))
print(model3.score(x_cv,np.ravel(y_cv)))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('VotingRegressor.csv',index=False)

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import StackingRegressor
model3 = StackingRegressor(estimators=[('lr', GradientBoostingRegressor(loss='lad', n_estimators=100,criterion='mae')), ('rf', KernelRidge(alpha=1, kernel='poly', degree= 4, coef0=2)),('sf',ExtraTreesRegressor(n_estimators=100, max_depth=None, criterion='mae'))],final_estimator=RandomForestRegressor(n_estimators=10))
model3.fit(x_train,np.ravel(scaled_y))
model3.score(x_cv,np.ravel(y_cv))
y_predict= pd.DataFrame(scaler.inverse_transform(model3.predict(x_predict).reshape(-1,1)),columns=['SalePrice'])
y_predict['Id']=test['Id'].to_numpy()
y_predict=y_predict[['Id','SalePrice']]
y_predict.to_csv('StackingRegressor.csv',index=False)

In [ ]:
type(y_training.ravel())

In [36]:
import lightgbm as lgb
train_data = lgb.Dataset(x_training, label=y_training.ravel())
valid_data = lgb.Dataset(x_cv, label=y_cv.ravel())
parameters = {
   
             'metric': 'rmse',
            
}
num_round = 10
bst = lgb.train(parameters, train_data, valid_sets=[valid_data],num_boost_round=5000,  early_stopping_rounds=100, valid_names=['valid'])
y =scaler.inverse_transform( (bst.predict(x_predict)).reshape(-1,1))
output = pd.DataFrame({'Id': test['Id'], 'SalePrice':y.ravel()})
output.to_csv("submission.csv", index=False)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3154
[LightGBM] [Info] Number of data points in the train set: 1022, number of used features: 67
[LightGBM] [Info] Start training from score 0.504716
[1]	valid's rmse: 0.266695
Training until validation scores don't improve for 100 rounds
[2]	valid's rmse: 0.244948
[3]	valid's rmse: 0.226716
[4]	valid's rmse: 0.210022
[5]	valid's rmse: 0.194932
[6]	valid's rmse: 0.181238
[7]	valid's rmse: 0.169286
[8]	valid's rmse: 0.158472
[9]	valid's rmse: 0.149458
[10]	valid's rmse: 0.141217
[11]	valid's rmse: 0.134308
[12]	valid's rmse: 0.127924
[13]	valid's rmse: 0.122423
[14]	valid's rmse: 0.117424
[15]	valid's rmse: 0.112985
[16]	valid's rmse: 0.109058
[17]	valid's rmse: 0.105292
[18]	valid's rmse: 0.102227
[19]	valid's rmse: 0.0998931
[20]	v

In [27]:
y.ravel()

array([127000.        , 145000.        , 186887.27344323, ...,
       154401.05277098, 122603.66204993, 228950.31657404])